<a href="https://colab.research.google.com/github/omardelarosa/website/blob/ai-thumbnails/python/AI_Thumbnails_using_stable_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --quiet --upgrade diffusers transformers scipy mediapy

In [ ]:
!huggingface-cli login

In [ ]:
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler

scheduler = PNDMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", skip_prk_steps=True)
# scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear")
# scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear")

In [ ]:
import mediapy as media
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline

model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda"
remove_safety = False


pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16, revision="fp16", use_auth_token=True)
if remove_safety:
  pipe.safety_checker = lambda images, clip_input: (images, False)
pipe = pipe.to(device)



In [ ]:
import json

input_file_name = 'prompts.json'

prompts = None
# Read the file data
with open(input_file_name, 'r') as f:
    prompts = json.load(f)

print(prompts)

In [ ]:
import pathlib
from tqdm import tqdm

num_images_per_prompt = 4

for prompt_idx, prompt_source_dict in tqdm(enumerate(prompts)):
  print(prompt)
  print(prompt_source_dict)

  style_suffix = " pixelated, sharp pixels with no border, pixel art, 8bit"

  prompt = prompt_source_dict["prompt"] + style_suffix
  outdir = prompt_source_dict["outdir"]

  # Create the outdir path for exporting
  pathlib.Path(outdir).mkdir(parents=True, exist_ok=True)

  _prompts = [ prompt ] * num_images_per_prompt
  with autocast("cuda"):
      results = pipe(_prompts, guidance_scale=7.5, num_inference_steps=75)
      images = results.images
  
  media.show_images(images)
  for idx, img in enumerate(images):
    img.save(outdir + f"/thumbnail_{idx}.jpg")